# Resizing Image

In [153]:
import cv2
import numpy as np

img = cv2.imread('lambo.png')
print(img.shape)

imgResize = cv2.resize(img,(300,200))

cv2.imshow("original",img)
cv2.imshow("Resized Image",imgResize)
cv2.waitKey()
cv2.destroyAllWindows()

(462, 623, 3)


## Drawing line, circle, Rectangle, text

In [165]:
import cv2
import numpy as np

img = np.zeros((512,512,3),np.uint8)
print(img.shape)

cv2.line(img,(0,0),(img.shape[1],img.shape[0]), (0,0,255),3)
cv2.rectangle(img, (0,0),(250,250), (0,0,255),2)
cv2.circle(img, (125,125), 80, (0,0,255),2)
cv2.putText(img, "Rajesh", (250,100), cv2.FONT_HERSHEY_TRIPLEX,2, (0,255,0),2)

cv2.imshow("Image",img)
cv2.waitKey()
cv2.destroyAllWindows()

(512, 512, 3)


## Warp Perspective

In [174]:
import cv2
import numpy as np

img = cv2.imread('cards.jpg')

width, height = 250, 350

pts1 = np.float32([[111,219], [287,188], [154,482], [352,440]])
pts2 = np.float32([[0,0], [width,0], [0,height],[width,height]])

matrix = cv2.getPerspectiveTransform(pts1,pts2)
imgOutput = cv2.warpPerspective(img, matrix, (width,height))

cv2.imshow('Image', img)
cv2.imshow('Output Image', imgOutput)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Contours / Shape Detection

In [93]:
# contours / shape detection
import cv2
import numpy as np

def getContours(img):
    # Finding How many countors(objects) in a image
    contours, heirarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # print(area)
        # This is just simply if the area is grreater than 300 then drawing contours
        if area > 1000:
            # -1 indicated draw all the contours
            cv2.drawContours(imgContour,cnt,-1,(255,255,0),3)
            # True is to determine it's a closesd contor
            # finding the curve length used to approximated the edges(corners)
            peri = cv2.arcLength(cnt,True)
            # Approximating how many cornor points we have
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            # print(approx) -- it will give the corner points of each shape
            # len(approx) -- if one of the shape having 3 corners then we will determine it as Rectangle
            objCor = len(approx)
            
            # Getting the boundary boxes of each object and
            # Drawing a bounding box at each of the object detected
            x,y,w,h = cv2.boundingRect(approx)
            if objCor == 3: objectType = 'Tri'
            elif objCor == 4:
                aspRatio = w/float(h)
                if aspRatio > 0.95 and aspRatio < 1.05: objectType = "Square"
                else: objectType = "Rectangle"
            elif objCor > 4: objectType = "circle"
            else: objectType = "None"
            cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0,255,0),2)
            # From these bounding boxes we can get the total width, height and center point of the object
            cv2.putText(imgContour,objectType, 
                        (x+(w//2)-10, y+(h//2)-10),
                        cv2.FONT_HERSHEY_COMPLEX,
                        0.5, (0,0,0),2)

img = cv2.imread('shapes.png')
imgContour = img.copy()

# First strp is to preprocess the image
# convert the image to gray scale and then find the edges
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Detecting the Edges
imgCanny = cv2.Canny(imgGray,50,50)

getContours(imgCanny)

cv2.imshow("Original",img)
cv2.imshow("Gray image",imgGray)
cv2.imshow("Canny image",imgCanny)
cv2.imshow("Contour image",imgContour)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Document Scanner

In [13]:
import cv2
import numpy as npf

widthImg = 640
heightImg = 480
cap = cv2.VideoCapture(0)
cap.set(3, widthImg)
cap.set(4, heightImg)
cap.set(10, 130)


def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur, 50,50)
    kernel = np.ones((2,2),np.uint8)
    imgDial = cv2.dilate(imgCanny,kernel, iterations=2)
    imgThres = cv2.erode(imgDial, kernel, iterations=1)
    return imgThres

def getContours(img):
    biggest = np.array([])
    maxArea = 0
    contours, heir = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # print(area)
        if area > 1000:
            cv2.drawContours(imgContour, cnt, -1, (255,0,0),3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            if area > maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
    cv2.drawContours(imgContour, biggest, -1, (255,0,0),20)
    return biggest

while True:
    success, img = cap.read()
    cv2.resize(img, (widthImg, heightImg))
    imgContour = img.copy()
    
    imgThres = preProcessing(img)
    biggest = getContours(imgThres)
    
    
    cv2.imshow("Result", imgContour)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

# Noise Removal

In [2]:
import cv2
import numpy as np

img = cv2.imread('early_1800.jpg')
imgBlur = cv2.GaussianBlur(img, (5,5),1)
blur = cv2.blur(img,(5,5))

cv2.imshow("Original",img)
cv2.imshow("Blur Image",imgBlur)
cv2.imshow("Blur",blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
import cv2
import numpy as np

img = cv2.imread('balloons_noisy.png')
averaging = cv2.blur(img,(5,5))
gaussina = cv2.GaussianBlur(img, (5,5),1)
median = cv2.medianBlur(img,5)

cv2.imshow("Original",img)
cv2.imshow("averagring",averaging)
cv2.imshow("Gaussian",gaussina)
cv2.imshow("Median",median)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [92]:
import cv2
import numpy as np

img = cv2.imread('balloons_noisy.png')
averaging = cv2.blur(img,(5,5))
gaussina = cv2.GaussianBlur(img, (5,5),1)
median = cv2.medianBlur(img,5)
bilateral = cv2.bilateralFilter(img, 5, 75,75) # 75 & 75 is just to keep the edges

cv2.imshow("Original",img)
# cv2.imshow("averagring",averaging)
# cv2.imshow("Gaussian",gaussina)
cv2.imshow("Median",median)
# cv2.imshow("bilateral",bilateral)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Thresholding

In [7]:
import cv2 as cv

img = cv.imread('gradient.jpg',0)
# Binary have only 1 and 0 that means pixel less than thres = 0 else 1
ret,thresh1 = cv.threshold(img,127,255,cv.THRESH_BINARY)
ret,thresh2 = cv.threshold(img,127,255,cv.THRESH_BINARY_INV)

# ThresThe pixel value upto threshold wont change and after threshold, if the pixel is greater thaan or lower it will change to threshold value
ret,thresh3 = cv.threshold(img,127,255,cv.THRESH_TRUNC)

# pixel less than trun = 0 and if greater than trun it remains same
ret,thresh4 = cv.threshold(img,127,255,cv.THRESH_TOZERO)
ret,thresh5 = cv.threshold(img,127,255,cv.THRESH_TOZERO_INV)

cv2.imshow("original",img)
cv2.imshow("Binary",thresh1)
cv2.imshow("Binary Inverse",thresh2)
cv2.imshow("TRUN",thresh3)
cv2.imshow("TOZERO",thresh4)
cv2.imshow("TOZERO INVERSE",thresh5)
cv2.waitKey()
cv2.destroyAllWindows()

# Morphological Transformation
### Erosion, Dilation, Opening & Closing
Opening: Erosion -> Dilation <br>
Closing: Dilation -> Erosion

In [81]:
import cv2 as cv
import numpy as np

kernel = np.ones((9,3),np.uint8)
kernel1 = np.ones((3,3),np.uint8)

img = cv.imread('T.png')
erosion = cv.erode(img, kernel, iterations=1)
dilation = cv.dilate(img, kernel, iterations=1)
# opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
# closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
# final = cv.erode(dilation, kernel, iterations=1)

# averaging = cv.blur(final,(5,5))
# gaussina = cv.GaussianBlur(final, (5,5),1)
# median = cv.medianBlur(final,5)
# bilateral = cv.bilateralFilter(final, 5, 75,75) 

# cv.imshow("average", averaging)
# cv.imshow("gaussina", gaussina)
# cv.imshow("median", median)
# cv.imshow("bilateral", bilateral)


cv.imshow("Original", img)
cv.imshow("Dilation", dilation)
# cv.imwrite("temp/Dilation39.png", dilation)
cv.imshow("Erosion", erosion)
# cv.imshow("final", final)
# cv.imshow("opening", opening)
# cv.imshow("closing", closing)
cv.waitKey()
cv.destroyAllWindows()

# Line Removal

In [5]:
import cv2 as cv
import numpy as np

v_kernel = np.ones((3,9),np.uint8)
h_kernel = np.ones((9,3),np.uint8)
# kernel1 = np.ones((3,3),np.uint8)

img = cv.imread('table.jpg')
# erosion = cv.erode(img, kernel, iterations=1)
dilation1 = cv.dilate(img, v_kernel, iterations=1)
dilation2 = cv.dilate(img, h_kernel, iterations=1)
# opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
# closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
# final = cv.erode(dilation, kernel, iterations=1)

# averaging = cv.blur(final,(5,5))
# gaussina = cv.GaussianBlur(final, (5,5),1)
# median = cv.medianBlur(final,5)
# bilateral = cv.bilateralFilter(final, 5, 75,75) 

# cv.imshow("average", averaging)
# cv.imshow("gaussina", gaussina)
# cv.imshow("median", median)
# cv.imshow("bilateral", bilateral)


cv.imshow("Original", img)
cv.imshow("Horizontal Lines", dilation1)
cv.imshow("Vertical Lines", dilation2)
# cv.imwrite("temp/Dilation39.png", dilation)
# cv.imshow("Erosion", erosion)
# cv.imshow("final", final)
# cv.imshow("opening", opening)
# cv.imshow("closing", closing)
cv.waitKey()
cv.destroyAllWindows()

In [109]:
# Image: table.jpg
import cv2 as cv
import numpy as np

v_kernel = np.ones((3,9),np.uint8)
h_kernel = np.ones((9,3),np.uint8)

img = cv.imread('table.jpg')
# erosion = cv.erode(img, kernel, iterations=1)
dilation1 = cv.dilate(img, v_kernel, iterations=1)
dilation2 = cv.dilate(img, h_kernel, iterations=1)

cv.imshow("Original", img)
cv.imshow("Horizontal Lines", dilation1)
cv.imshow("Vertical Lines", dilation2)
cv.waitKey()
cv.destroyAllWindows()

# Bounding Boxes (IEEE Paper)

In [14]:
import cv2 as cv
import numpy as np

img = cv.imread("pdf_page.jpg")
imgContour = img.copy()
imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
blur = cv.GaussianBlur(imgGray, (5,5),0)
# thres = cv.threshold(blur, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)[1]

bit = cv.bitwise_not(blur)
kernel = np.ones((5,5),np.uint8)

dilate = cv.dilate(bit, kernel, iterations=1)

contour = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)[0]
for cnt in contour:
    peri = cv.arcLength(cnt,True)
    approx = cv.approxPolyDP(cnt, 0.02*peri, True)
    x,y,w,h = cv.boundingRect(approx)
    if w > 50 and h > 100:
        cv.rectangle(imgContour, (x,y), (x+w,y+h), (0,255,0),2)
    # cv.drawContours(imgContour, cnt, -1, (255,0,0),3)

# cv.imshow("Image",img)
# cv.imshow("Blur",blur)
cv.imshow("bit",dilate)
cv.imshow("Contour",imgContour)
cv.waitKey()
cv.destroyAllWindows()

# Text Detection From Image
### Detecting Characters

In [149]:
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
img = cv2.imread('text.png')

h_img, w_img,_ = img.shape

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# It will detect the text and return it (not position)
# print(pytesseract.image_to_string(imgGray))

# it will give the eachh word with x,y,w,h position
boxes = pytesseract.image_to_boxes(imgGray)
for box in boxes.splitlines():
    b = box.split(' ')
    x,y,w,h = int(b[1]),int(b[2]),int(b[3]),int(b[4])
    # print(x,y,w,h)
    cv2.rectangle(img, (x,h_img-y), (w,h_img-h), (0,0,255),2)
    cv2.putText(img, b[0],(x,h_img-y+20),cv2.FONT_HERSHEY_COMPLEX,0.5,(50,50,255),1)
    


cv2.imshow("Image",img)
# cv2.imshow("Image",imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

### Detecting Words

In [148]:
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
img = cv2.imread('text.png')

h_img, w_img,_ = img.shape

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# It will detect the text and return it (not position)
# print(pytesseract.image_to_string(imgGray))

# it will give the each character with x,y,w,h position
boxes = pytesseract.image_to_data(imgGray)
print(boxes)
for i,box in enumerate(boxes.splitlines()):
    if i != 0: # Eliminating the Header
        b = box.split()
        if len(b) == 12:
            x,y,w,h = int(b[6]),int(b[7]),int(b[8]),int(b[9])
            # print(x,y,w,h)
            cv2.rectangle(img, (x,y), (w+x,h+y), (0,0,255),2)
            cv2.putText(img, b[11],(x,y),cv2.FONT_HERSHEY_COMPLEX,0.5,(50,50,255),1)
            


cv2.imshow("Image",img)
# cv2.imshow("Image",imgGray)
cv2.waitKey()
cv2.destroyAllWindows()

level	page_num	block_num	par_num	line_num	word_num	left	top	width	height	conf	text
1	1	0	0	0	0	0	0	742	347	-1	
2	1	1	0	0	0	164	66	377	40	-1	
3	1	1	1	0	0	164	66	377	40	-1	
4	1	1	1	1	0	164	66	377	40	-1	
5	1	1	1	1	1	164	66	171	40	96.429344	RAJESH
5	1	1	1	1	2	355	67	186	38	96.163536	VARMA
2	1	2	0	0	0	73	217	596	40	-1	
3	1	2	1	0	0	73	217	596	40	-1	
4	1	2	1	1	0	73	217	596	40	-1	
5	1	2	1	1	1	73	217	596	40	0.000000	1234567891011
5	1	2	1	1	2	602	213	67	61	0.000000	12

